マウスの総移動距離を算出

In [ ]:
import os
import math

def calculate_distance(file_path):
    """
    指定されたファイルからマウスの総移動距離を計算する
    """
    total_distance = 0.0
    last_position = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # マウスの移動に関する行を抽出
            if line.startswith("MOUSE_MOVED") or line.startswith("MOUSE_DRAGGED"):
                try:
                    parts = line.strip().split(":")
                    x, y = map(int, parts[-1].strip().strip("()").split(","))
                    current_position = (x, y)

                    if last_position is not None:
                        # 前回の位置からの距離を計算
                        distance = math.sqrt((current_position[0] - last_position[0])**2 +
                                             (current_position[1] - last_position[1])**2)
                        total_distance += distance

                    last_position = current_position
                except Exception as e:
                    print(f"Error processing line: {line}. Error: {e}")
    return total_distance

def process_logs(base_folder_path, output_file):
    """
    各フォルダのログファイルを処理し、総移動距離を計算して保存する
    """
    results = []

    for i in range(1, 26):  # フォルダ番号の範囲を指定（例: 1〜25）
        subject_number = f"{i:02d}"
        folder_path = os.path.join(base_folder_path, subject_number, "logs")

        if os.path.exists(folder_path):
            log_files = [file for file in os.listdir(folder_path) if file.startswith("ui-log") and file.endswith(".txt")]
            # 日付の昇順でソート
            log_files.sort()

            for index, file_name in enumerate(log_files):
                file_path = os.path.join(folder_path, file_name)
                total_distance = calculate_distance(file_path)

                # Taskの決定
                if index == 0:
                    task = "training"
                else:
                    task = f"task_{index:02d}"

                results.append((subject_number, file_name, task, total_distance))

    # 結果をファイルに保存
    with open(output_file, 'w', encoding='utf-8') as output:
        output.write("Subject,File,Total Distance\n")
        for subject, file_name, task, distance in results:
            output.write(f"{subject},{file_name},{task},{distance:.2f}\n")
            
    print(f"結果を {output_file} に保存しました。")

# ベースフォルダのパスと出力ファイル名を指定
base_folder_path = r".\SubjectData"
output_file = "mouse_distances.csv"

process_logs(base_folder_path, output_file)


ValueError: too many values to unpack (expected 3)